In [2]:
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import requests as rq
import os
import chardet
import re
import plotly.graph_objects as go

pd.options.display.max_colwidth = 1000

# IEG-M

In [2]:
DATA_PATH = '../data'

In [35]:
df_iegm = pd.read_csv(os.path.join(DATA_FOLDER,'iegm.csv'), delimiter=';', encoding='Windows-1252') 

/home/andrew/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [49]:
df_iegm.head()

,cod_ibge,municipio,porte,ano_exercicio,indice,cod_pergunta,texto_pergunta_genérica,cod_resposta,rotulo_resposta,valor_resposta
0,3500204,Adolfo,PP,2014.0,i-Amb,M05Q00100,"Sobre o Plano Municipal de Gestão Integrada de Resíduos Sólidos (PMGIRS) – Lei nº 12.305/2010 –, informe:",M05Q00100R00100,instrumento normativo publicado ou promulgado,instrumento normativo publicado ou promulgado
1,3501202,Álvares Florence,PP,2014.0,i-Amb,M05Q00100,"Sobre o Plano Municipal de Gestão Integrada de Resíduos Sólidos (PMGIRS) – Lei nº 12.305/2010 –, informe:",M05Q00100R00300,estão em outras fases de elaboração,está em outras fases de elaboração
2,3500303,Aguaí,M,2014.0,i-Amb,M05Q00100,"Sobre o Plano Municipal de Gestão Integrada de Resíduos Sólidos (PMGIRS) – Lei nº 12.305/2010 –, informe:",M05Q00100R00400,não realizaram o plano,não realizou o plano
3,3500402,Águas da Prata,P,2014.0,i-Amb,M05Q00100,"Sobre o Plano Municipal de Gestão Integrada de Resíduos Sólidos (PMGIRS) – Lei nº 12.305/2010 –, informe:",M05Q00100R00100,instrumento normativo publicado ou promulgado,instrumento normativo publicado ou promulgado
4,3500501,Águas de Lindóia,P,2014.0,i-Amb,M05Q00100,"Sobre o Plano Municipal de Gestão Integrada de Resíduos Sólidos (PMGIRS) – Lei nº 12.305/2010 –, informe:",M05Q00100R00300,estão em outras fases de elaboração,está em outras fases de elaboração


In [53]:
df_iegm.where(df_iegm['rotulo_resposta'] != df_iegm['valor_resposta']).dropna().head()

,cod_ibge,municipio,porte,ano_exercicio,indice,cod_pergunta,texto_pergunta_genérica,cod_resposta,rotulo_resposta,valor_resposta
1,3501202,Álvares Florence,PP,2014.0,i-Amb,M05Q00100,"Sobre o Plano Municipal de Gestão Integrada de Resíduos Sólidos (PMGIRS) – Lei nº 12.305/2010 –, informe:",M05Q00100R00300,estão em outras fases de elaboração,está em outras fases de elaboração
2,3500303,Aguaí,M,2014.0,i-Amb,M05Q00100,"Sobre o Plano Municipal de Gestão Integrada de Resíduos Sólidos (PMGIRS) – Lei nº 12.305/2010 –, informe:",M05Q00100R00400,não realizaram o plano,não realizou o plano
4,3500501,Águas de Lindóia,P,2014.0,i-Amb,M05Q00100,"Sobre o Plano Municipal de Gestão Integrada de Resíduos Sólidos (PMGIRS) – Lei nº 12.305/2010 –, informe:",M05Q00100R00300,estão em outras fases de elaboração,está em outras fases de elaboração
8,3500758,Alambari,P,2014.0,i-Amb,M05Q00100,"Sobre o Plano Municipal de Gestão Integrada de Resíduos Sólidos (PMGIRS) – Lei nº 12.305/2010 –, informe:",M05Q00100R00400,não realizaram o plano,não realizou o plano
10,3500907,Altair,PP,2014.0,i-Amb,M05Q00100,"Sobre o Plano Municipal de Gestão Integrada de Resíduos Sólidos (PMGIRS) – Lei nº 12.305/2010 –, informe:",M05Q00100R00300,estão em outras fases de elaboração,está em outras fases de elaboração


In [54]:
filter_cidade = df_iegm['indice'] == 'i-Cidade'

In [56]:
df_iegm.where(filter_cidade).dropna().head()

,cod_ibge,municipio,porte,ano_exercicio,indice,cod_pergunta,texto_pergunta_genérica,cod_resposta,rotulo_resposta,valor_resposta
17323,3500758,Alambari,P,2014.0,i-Cidade,M06Q00100,O município possui a Coordenadoria Municipal de Defesa Civil - COMDEC estruturada? (Lei nº 12.608/2012)?,M06Q00100R00200,estão em fase de estruturação,está em fase de estruturação
17324,3500204,Adolfo,PP,2014.0,i-Cidade,M06Q00100,O município possui a Coordenadoria Municipal de Defesa Civil - COMDEC estruturada? (Lei nº 12.608/2012)?,M06Q00100R00300,não possuem,não
17325,3500204,Adolfo,PP,2014.0,i-Cidade,M06Q00200,A prefeitura municipal possui local físico com sala e telefone para atendimento de ocorrências de Defesa Civil?,M06Q00200R00200,não possuem local físico,não
17326,3500204,Adolfo,PP,2014.0,i-Cidade,M06Q00300,O município está cadastrado no sistema da Defesa Civil estadual denominado SIDEC (Sistema Integrado de Defesa Civil)?,M06Q00300R00200,não possuem,não
17327,3500204,Adolfo,PP,2014.0,i-Cidade,M06Q00400,O município utiliza alguma forma de registro eletrônico para cadastramento de ocorrência de Defesa Civil?,M06Q00400R00200,não possuem,não


# Transparência

In [ ]:
def get_df_from_endpoint(tipo, municipio, exercicio, mes):
    r = rq.get('https://transparencia.tce.sp.gov.br/api/json/{}/{}/{}/{}'\
               .format(tipo, municipio, exercicio, mes))
    if r.status_code == 200:
        return pd.DataFrame(r.json())
    else:
        raise Exception('Bad Request')

In [88]:
df_adolfo = get_df_from_endpoint('despesas','adolfo',2015,1)

In [91]:
df_adolfo.head()

,dt_emissao_despesa,evento,id_fornecedor,mes,nm_fornecedor,nr_empenho,orgao,vl_despesa
0,12/01/2015,Empenhado,CNPJ - PESSOA JURÍDICA - 01704233000138,Janeiro,FIORILLI SOC.CIVIL LTDA.,134-2015,PREFEITURA MUNICIPAL DE ADOLFO,"5869,68"
1,20/01/2015,Empenhado,IDENTIFICAÇÃO ESPECIAL - SEM CPF/CNPJ - 79,Janeiro,BANCO DO BRASIL,194-2015,PREFEITURA MUNICIPAL DE ADOLFO,"124,80"
2,13/01/2015,Empenhado,CNPJ - PESSOA JURÍDICA - 06291846000104,Janeiro,CONSTROESTE CONSTRUTORA E PARTICIPACOES LTDA,147-2015,PREFEITURA MUNICIPAL DE ADOLFO,"689,00"
3,07/01/2015,Empenhado,IDENTIFICAÇÃO ESPECIAL - SEM CPF/CNPJ - 2272,Janeiro,TRIBUNAL DE JUSTICA DO ESTADO DE SAO PAULO,114-2015,PREFEITURA MUNICIPAL DE ADOLFO,"14242,04"
4,20/01/2015,Empenhado,CNPJ - PESSOA JURÍDICA - 02558157000162,Janeiro,TELEFONICA BRASIL S.A,190-2015,PREFEITURA MUNICIPAL DE ADOLFO,"824,32"


In [90]:
df_adolfo['nm_fornecedor'].unique()

array(['FIORILLI SOC.CIVIL LTDA.', 'BANCO DO BRASIL',
       'CONSTROESTE CONSTRUTORA E PARTICIPACOES LTDA',
       'TRIBUNAL DE JUSTICA DO ESTADO DE SAO PAULO',
       'TELEFONICA BRASIL S.A', 'EMP. BRAS. DE CORREIOS E TELEGRAFOS',
       'CONFEDERACAO NACIONAL DE MUNICIPIOS', 'MELAINE REGINA MARTINEZ',
       'INST.NAC.PREV.SOCIAL', 'IMPRENSA OFICIAL DO ESTADO S/A - IMESP',
       'CAIXA ECONOMICA FEDERAL', 'NUTRICIONALE COM.DE ALIMENTOS LTDA.',
       'LARISSA BEJARANO RECHE', 'CAMILA MORAES BENATTI',
       'ECAD-ESCRIT.CENT.ARRECADACAO E DISTRIBUICAO',
       'CIA. NACIONAL DE ENERGIA ELETRICA',
       'JOSE NILTON DE OLIVEIRA DIVERSOES-ME', 'COP FAC MAQUINAS LTDA',
       'SABESP-COMP.SAN.BASICO ESTADO SP',
       'LABORATORIO ANAL.CLINICAS GALLI LTDA',
       'AUTO POSTO LARANJEIRAS DE MENDONCA LTDA EPP',
       'TEK CAMBIO JOSE BONIFACIO LTDA -ME', 'FRANCINALDO GOMES DA SILVA',
       'RAP APARECIDA COMERCIO DE MEDICAMENTO LTDA ME',
       'COMERCIAL CIRURGICA RIOCLARENSE LTDA'

http://dados.gov.br/noticia/casa-civil-abre-consulta-publica-sobre-inventario-de-dados-abertos

# IBGE

## Observações

Existem duas APIs relevantes para obtenção de informações do IBGE, a saber: a API de Agregados e a API de Pesquisas.

A API de Pesquisas fornece um resumo dos dados disponíveis através da API de agregados. Os dados dessa API são gerados de forma automatizada a partir dos dados disponibilizados pela API de Agregados.

In [55]:
def get_df_from_endpoint(url, args=[], normalize=False, drop='notas'):
    """Returns a dataframe from given endpoint
    
    Parameters
    ----------
    url : str
        URL for the endpoint
    args : array, optional
        By default, 'args' is an empty array. For those endpoints which
        parameters are required, you must place them in this argument
        in the same order as they would appear in the URL
    normalize : bool, optional
        Whether the returned json should be normalized or not (default is False)
    drop : string, optional
        Column names to be droped. By default tries dropping 'notas'

    Returns
    -------
    DataFrame
        The DataFrame created from the json response string
    """
    
    r = rq.get(url.format(*args))
    if r.status_code == 200:
        if normalize:
            df = json_normalize(r.json(),['res'], meta=['id'])
            
            try:
                df = df.drop(columns=drop)
            except:
                pass
            
        else:
            df = pd.DataFrame(r.json())
        return df
    else:
        raise Exception('Bad Request for url\t{}'.format(url.format(*args)))

## Agregados

In [6]:
url = 'https://servicodados.ibge.gov.br/api/v3/agregados'
df_agregados = get_df_from_endpoint(url)

In [7]:
for iden,nome in df_agregados[['id','nome']].values:
    print('{}\t{}'.format(iden,nome))

CL	Cadastro Central de Empresas
CA	Censo Agropecuário
ME	Censo Comum do Mercosul, Bolívia e Chile
CD	Censo Demográfico
CM	Contagem da População 
SU	Contas Nacionais Anuais
ST	Contas Nacionais Trimestrais
DE	Demografia das Empresas e Estatísticas de Empreendedorismo
RC	Estatísticas do Registro Civil
XF	Estimativas de População
FP	Fundações Privadas e Associações Sem Fins Lucrativos
IU	Indicadores de Desenvolvimento Sustentável
IL	Índice de Preços ao Consumidor em Real
IR	Índice de Preços ao Produtor
IJ	Índice de Reajuste do Salário Mínimo
PC	Índice Nacional de Preços ao Consumidor
IA	Índice Nacional de Preços ao Consumidor Amplo
IQ	Índice Nacional de Preços ao Consumidor Amplo 15
IG	Índice Nacional de Preços ao Consumidor Amplo Especial
PG	Índice Nacional de Preços ao Consumidor Especial
LA	Levantamento Sistemático da Produção Agrícola
C2	Objetivos de Desenvolvimento Sustentável
AC	Pesquisa Anual da Indústria da Construção
PB	Pesquisa Anual de Comércio
PY	Pesquisa Anual de Serviços
PP	P

In [12]:
agregados_relevantes = ['IU', 'C2', 'AM', 'IT', 'OF', 'SH', 'PT', 'PE', 
                        'SB', 'SI', 'PD', 'AA', 'XN', 'SC', 'PJ']

## Pesquisas

In [8]:
url = 'https://servicodados.ibge.gov.br/api/v1/pesquisas'
df_pesquisas = get_df_from_endpoint(url)

In [9]:
for iden,nome in df_pesquisas[['id','nome']].values:
    print('{}\t{}'.format(iden,nome))

1	MUNIC - Perfil dos Municípios Brasileiros
11	SNIG - Pesquisa Nacional de Informação de Gênero
13	Censo escolar - sinopse
14	Produção Agrícola - Lavoura Temporária
15	Produção Agrícola - Lavoura Permanente
16	Extração vegetal e Silvicultura
17	Morbidade
18	Pecuária
19	Cadastro Central de Empresas
20	Registro civil
21	Finanças públicas
22	Frota de veículos
23	Censo
24	Censo Agropecuário
29	Instituições financeiras
30	Pesquisa nacional de saneamento básico
31	Produção agrícola - Cereais, leguminosas e oleaginosas
32	Serviços de saúde
33	Sinopse municipal
34	Pesquisa de serviços de hospedagem
35	Fundações privadas e associações sem fins lucrativos
36	Mapa de pobreza e desigualdade
37	Índice de Desenvolvimento Humano
38	Produto Interno Bruto dos Municípios
39	Taxa de mortalidade infantil
40	Índice de Desenvolvimento da Educação Básica
42	Notificações de dengue registradas
43	Censo - Séries históricas
44	PNAD - Pesquisa Nacional por Amostra de Domicílios
45	Síntese de Indicadores Sociais
4

## Recuperando id dos municipios de SP

In [4]:
def get_mun_dict(estado, key='nome'):
    # Check arg
    if key != 'nome' and key != 'id':
        raise Exception("Bad value for keyword 'key'. Expected 'id' or 'nome', got '{}'.".format(key))
        
    url = 'https://servicodados.ibge.gov.br/api/v1/localidades/estados'
    df_loc = get_df_from_endpoint(url)
    
    # Check arg
    if estado not in df_loc['nome'].unique():
        raise Exception("Bad value for keyword for 'estado'. Expected a valid state name, got '{}'.".format(estado))
    
    id_estado = df_loc[df_loc['nome'] == estado]['id'].values[0]
    
    url = 'https://servicodados.ibge.gov.br/api/v1/localidades/estados/{}/municipios'
    df_mun = get_df_from_endpoint(url, [id_estado])
    mun_dict = df_mun[['id','nome']].to_dict(orient='list')
    
    if key == 'id':
        mun_dict = dict([mun_dict['id'][i], mun_dict['nome'][i]] for i in range(len(mun_dict['id'])))
    else:
        mun_dict = dict([mun_dict['nome'][i], mun_dict['id'][i]] for i in range(len(mun_dict['id'])))

    return mun_dict

In [5]:
mun_to_id = get_mun_dict('São Paulo', key='nome')

In [6]:
id_to_mun = get_mun_dict('São Paulo', key='id')

In [7]:
mun_to_id['Adamantina']

3500105

In [8]:
id_to_mun[3500105]

'Adamantina'

## Filtrando contexto das pesquisas

In [9]:
def filter_municipio(s):
    """Function for filtering municipios
    
    This function can be applyed to a pandas dataframe
    in order to generate a True/False list. This list,
    in turn, can be used for filtering the rows from
    this given dataframe.
    """
    
    re_municipio = re.compile(r"(0|1)*1(1|0)\b")
    if s:
        return True if re_municipio.match(s) else False
    else:
        return False

def filter_pesquisa(df, by='municipio'):
    if by == 'municipio':
        filter_by = df['contexto'].apply(filter_municipio)    
    else:
        raise Exception("Bad value for keyword 'key'")
    return df[filter_by]

In [10]:
url = 'https://servicodados.ibge.gov.br/api/v1/pesquisas'
df_pes = get_df_from_endpoint(url)

In [11]:
df_pes_mun = filter_pesquisa(df_pes)
df_pes_mun[['id','nome','contexto']]

,id,nome,contexto
0,1,MUNIC - Perfil dos Municípios Brasileiros,10
2,13,Censo escolar - sinopse,101010
3,14,Produção Agrícola - Lavoura Temporária,101010
4,15,Produção Agrícola - Lavoura Permanente,101010
5,16,Extração vegetal e Silvicultura,101010
6,17,Morbidade,1010
7,18,Pecuária,101010
8,19,Cadastro Central de Empresas,101010
9,20,Registro civil,101010
10,21,Finanças públicas,1010


In [50]:
pesquisas_relevantes = set([1, 13, 21, 22, 23, 30, 32, 33, 34, 35, 36, 
                        37, 38, 39, 40 ,42, 44, 45, 46, 47, 10054, 10058, 
                       10073,  10075, 10077, 10085, 10087])
pesquisas_disponiveis = set(df_pes_mun['id'].values)
id_pes = pesquisas_relevantes & pesquisas_disponiveis

### Obtendo resultados das pesquisas

Testando com Adamantina e pesquisa 10058.

In [12]:
id_pes = 10058
id_mun = mun_to_id['Adamantina']
url = 'https://servicodados.ibge.gov.br/api/v1/pesquisas/{}/resultados/{}'
df_val = get_df_from_endpoint(url,[id_pes,id_mun], normalize=True)

In [13]:
df_val.head()

,localidade,res,id
0,350010,"{'2010': '99.3', '2012-2014': None, '2014': None, '2015': None, '2016': None, '2017': None}",60029
1,350010,"{'2010': '95.8', '2012-2014': None, '2014': None, '2015': None, '2016': None, '2017': None}",60030
2,350010,"{'2010': '17.9', '2012-2014': None, '2014': None, '2015': None, '2016': None, '2017': None}",60031
3,350010,"{'2010': None, '2012-2014': None, '2014': '25229.43', '2015': None, '2016': None, '2017': None}",60047
4,350010,"{'2010': None, '2012-2014': None, '2014': None, '2015': '55.4', '2016': None, '2017': None}",60048


In [14]:
len(df_val)

13

### Desaninhando res

In [15]:
def expand_res(df, columns=['ano','valor','id','localidade']):
    df_un = pd.DataFrame(columns=columns)
    for index,row in df.iterrows():
        for k,v in row['res'].items():
            '''
            Em alguns casos, ano é um range, e.g. '2010-2014'
            Se esse for o caso, é preciso quebrar a string e
            adicionar uma linha para cada ano no intervalo
            '''
            if len(k) > 4:
                for k_n in range(int(k[:4]),int(k[5:])):
                    data = [k_n,None,row.loc['id'],row.loc['localidade']]
                    new_row = pd.DataFrame([data], columns=columns)
                    df_un = df_un.append(new_row)
            else:
                data = [k,v,row.loc['id'],row.loc['localidade']]
                new_row = pd.DataFrame([data], columns=columns)
                df_un = df_un.append(new_row)

    df = df.merge(df_un, on=['id','localidade']).drop(columns='res')
    return df

In [16]:
df_val = expand_res(df_val)

In [17]:
df_val.head()

,localidade,id,ano,valor
0,350010,60029,2010,99.3
1,350010,60029,2012,None
2,350010,60029,2013,None
3,350010,60029,2014,None
4,350010,60029,2015,None


In [18]:
len(df_val)

91

In [19]:
df_val.dropna(how='any')

,localidade,id,ano,valor
0,350010,60029,2010,99.3
7,350010,60030,2010,95.8
14,350010,60031,2010,17.9
24,350010,60047,2014,25229.43
32,350010,60048,2015,55.4
39,350010,60041,2015,6.6
46,350010,60042,2015,5
49,350010,60045,2010,97.6
61,350010,60032,2016,1.6
73,350010,60036,2014,33.3


## Explorando indicadores

In [20]:
url = 'https://servicodados.ibge.gov.br/api/v1/pesquisas/{}/indicadores/{}'
df_ind = get_df_from_endpoint(url,[10058,0])

In [21]:
df_ind.head()

,children,classe,id,indicador,nota,posicao
0,"[{'id': 60029, 'posicao': '1.1', 'indicador': 'Arborização de vias públicas', 'classe': 'I', 'unidade': {'id': '%', 'classe': 'N', 'multiplicador': 1}, 'children': [], 'nota': [{'periodo': '2010', 'notas': ['[domicílios urbanos em face de quadra com arborização/domicílios urbanos totais] x100']}], 'fonte': [{'periodo': '2010', 'fontes': ['IBGE, Censo Demográfico 2010']}]}, {'id': 60030, 'posicao': '1.2', 'indicador': 'Esgotamento sanitário adequado', 'classe': 'I', 'unidade': {'id': '%', 'classe': 'N', 'multiplicador': 1}, 'children': [], 'nota': [{'periodo': '2010', 'notas': ['[população total residente nos domicílios particulares permanentes com esgotamento sanitário do tipo rede geral e fossa séptica / População total residente nos domicílios particulares permanentes] x 100']}], 'fonte': [{'periodo': '2010', 'fontes': ['Ministério do Planejamento, Desenvolvimento e Gestão', 'IBGE, Instituto Brasileiro de Geografia e Estatística']}]}, {'id': 60031, 'posicao': '1.3', 'indicador': ...",T,60027,Ambiente,[],1
1,"[{'id': 60047, 'posicao': '2.1', 'indicador': 'PIB per capita', 'classe': 'I', 'unidade': {'id': 'R$', 'classe': '$', 'multiplicador': 1}, 'children': [], 'nota': [], 'fonte': [{'periodo': '-', 'fontes': ['IBGE, em parceria com os Órgãos Estaduais de Estatística, Secretarias Estaduais de Governo e Superintendência da Zona Franca de Manaus - SUFRAMA']}]}, {'id': 60048, 'posicao': '2.2', 'indicador': 'Percentual das receitas oriundas de fontes externas', 'classe': 'I', 'unidade': {'id': '%', 'classe': 'N', 'multiplicador': 1}, 'children': [], 'nota': [], 'fonte': [{'periodo': '2015', 'fontes': ['Secretaria do Tesouro Nacional (STN) - Balanço do Setor Público Nacional (BSPN) 2015']}]}]",T,60046,Economia,[],2
2,"[{'id': 60040, 'posicao': '3.1', 'indicador': 'Índice de Desenvolvimento da Educação Básica', 'classe': None, 'children': [{'id': 60041, 'posicao': '3.1.1', 'indicador': 'Anos iniciais do ensino fundamental', 'classe': 'I', 'children': [], 'nota': [], 'fonte': [{'periodo': '2015', 'fontes': ['MEC/INEP - Censo Escolar 2016']}]}, {'id': 60042, 'posicao': '3.1.2', 'indicador': 'Anos finais do ensino fundamental', 'classe': 'I', 'children': [], 'nota': [], 'fonte': [{'periodo': '2015', 'fontes': ['MEC/INEP - Censo Escolar 2016']}]}], 'nota': []}, {'id': 60043, 'posicao': '3.2', 'indicador': 'Taxa de escolarização', 'classe': None, 'children': [{'id': 60044, 'posicao': '3.2.1', 'indicador': 'Grupo de idade', 'classe': None, 'children': [{'id': 60045, 'posicao': '3.2.1.1', 'indicador': '6 a 14 anos de idade', 'classe': 'I', 'unidade': {'id': '%', 'classe': 'N', 'multiplicador': 1}, 'children': [], 'nota': [{'periodo': '2010', 'notas': ['[população residente no município de 6 a 14 anos de...",T,60039,Educação,[],3
3,"[{'id': 60032, 'posicao': '4.1', 'indicador': 'Internações por diarreia', 'classe': 'I', 'unidade': {'id': 'internações por mil habitantes', 'classe': 'N', 'multiplicador': 1}, 'children': [], 'nota': [{'periodo': '2016', 'notas': ['[número de internações por diarreia/população residente] x 1000']}], 'fonte': [{'periodo': '2016', 'fontes': ['Ministério da Saúde, DATASUS - Departamento de Informática do SUS', 'IBGE, Estimativas de população residente']}]}, {'id': 60034, 'posicao': '4.2', 'indicador': 'Mortalidade infantil', 'classe': None, 'children': [{'id': 60033, 'posicao': '4.2.1', 'indicador': 'Média', 'classe': 'I', 'unidade': {'id': 'por mil nascidos vivos', 'classe': 'N', 'multiplicador': 1}, 'children': [], 'nota': [{'periodo': '2012-2014', 'notas': ['média de óbitos de crianças antes de completar 1 ano / [médio de nascidos vivos / 1000]']}], 'fonte': [{'periodo': '2012-2014', 'fontes': ['Ministério da Saúde, DATASUS - Departamento de Informática do SUS']}]}], 'nota': []}]",T,60028,Saúde,[],4
4,"[{'id': 60036, 'posicao': '5.1', 'indicador': 'População ocupada', 'classe': 'I', 'unidade': {'id': '%', 'classe': 'N', 'multiplicador': 1}, 'children': [], 'nota': 

### Desaninhando indicadores

In [22]:
url = 'https://servicodados.ibge.gov.br/api/v1/pesquisas/{}/indicadores/0'
df_ind = get_df_from_endpoint(url, [id_pes])

In [23]:
df_ind

,children,classe,id,indicador,nota,posicao
0,"[{'id': 60029, 'posicao': '1.1', 'indicador': 'Arborização de vias públicas', 'classe': 'I', 'unidade': {'id': '%', 'classe': 'N', 'multiplicador': 1}, 'children': [], 'nota': [{'periodo': '2010', 'notas': ['[domicílios urbanos em face de quadra com arborização/domicílios urbanos totais] x100']}], 'fonte': [{'periodo': '2010', 'fontes': ['IBGE, Censo Demográfico 2010']}]}, {'id': 60030, 'posicao': '1.2', 'indicador': 'Esgotamento sanitário adequado', 'classe': 'I', 'unidade': {'id': '%', 'classe': 'N', 'multiplicador': 1}, 'children': [], 'nota': [{'periodo': '2010', 'notas': ['[população total residente nos domicílios particulares permanentes com esgotamento sanitário do tipo rede geral e fossa séptica / População total residente nos domicílios particulares permanentes] x 100']}], 'fonte': [{'periodo': '2010', 'fontes': ['Ministério do Planejamento, Desenvolvimento e Gestão', 'IBGE, Instituto Brasileiro de Geografia e Estatística']}]}, {'id': 60031, 'posicao': '1.3', 'indicador': ...",T,60027,Ambiente,[],1
1,"[{'id': 60047, 'posicao': '2.1', 'indicador': 'PIB per capita', 'classe': 'I', 'unidade': {'id': 'R$', 'classe': '$', 'multiplicador': 1}, 'children': [], 'nota': [], 'fonte': [{'periodo': '-', 'fontes': ['IBGE, em parceria com os Órgãos Estaduais de Estatística, Secretarias Estaduais de Governo e Superintendência da Zona Franca de Manaus - SUFRAMA']}]}, {'id': 60048, 'posicao': '2.2', 'indicador': 'Percentual das receitas oriundas de fontes externas', 'classe': 'I', 'unidade': {'id': '%', 'classe': 'N', 'multiplicador': 1}, 'children': [], 'nota': [], 'fonte': [{'periodo': '2015', 'fontes': ['Secretaria do Tesouro Nacional (STN) - Balanço do Setor Público Nacional (BSPN) 2015']}]}]",T,60046,Economia,[],2
2,"[{'id': 60040, 'posicao': '3.1', 'indicador': 'Índice de Desenvolvimento da Educação Básica', 'classe': None, 'children': [{'id': 60041, 'posicao': '3.1.1', 'indicador': 'Anos iniciais do ensino fundamental', 'classe': 'I', 'children': [], 'nota': [], 'fonte': [{'periodo': '2015', 'fontes': ['MEC/INEP - Censo Escolar 2016']}]}, {'id': 60042, 'posicao': '3.1.2', 'indicador': 'Anos finais do ensino fundamental', 'classe': 'I', 'children': [], 'nota': [], 'fonte': [{'periodo': '2015', 'fontes': ['MEC/INEP - Censo Escolar 2016']}]}], 'nota': []}, {'id': 60043, 'posicao': '3.2', 'indicador': 'Taxa de escolarização', 'classe': None, 'children': [{'id': 60044, 'posicao': '3.2.1', 'indicador': 'Grupo de idade', 'classe': None, 'children': [{'id': 60045, 'posicao': '3.2.1.1', 'indicador': '6 a 14 anos de idade', 'classe': 'I', 'unidade': {'id': '%', 'classe': 'N', 'multiplicador': 1}, 'children': [], 'nota': [{'periodo': '2010', 'notas': ['[população residente no município de 6 a 14 anos de...",T,60039,Educação,[],3
3,"[{'id': 60032, 'posicao': '4.1', 'indicador': 'Internações por diarreia', 'classe': 'I', 'unidade': {'id': 'internações por mil habitantes', 'classe': 'N', 'multiplicador': 1}, 'children': [], 'nota': [{'periodo': '2016', 'notas': ['[número de internações por diarreia/população residente] x 1000']}], 'fonte': [{'periodo': '2016', 'fontes': ['Ministério da Saúde, DATASUS - Departamento de Informática do SUS', 'IBGE, Estimativas de população residente']}]}, {'id': 60034, 'posicao': '4.2', 'indicador': 'Mortalidade infantil', 'classe': None, 'children': [{'id': 60033, 'posicao': '4.2.1', 'indicador': 'Média', 'classe': 'I', 'unidade': {'id': 'por mil nascidos vivos', 'classe': 'N', 'multiplicador': 1}, 'children': [], 'nota': [{'periodo': '2012-2014', 'notas': ['média de óbitos de crianças antes de completar 1 ano / [médio de nascidos vivos / 1000]']}], 'fonte': [{'periodo': '2012-2014', 'fontes': ['Ministério da Saúde, DATASUS - Departamento de Informática do SUS']}]}], 'nota': []}]",T,60028,Saúde,[],4
4,"[{'id': 60036, 'posicao': '5.1', 'indicador': 'População ocupada', 'classe': 'I', 'unidade': {'id': '%', 'classe': 'N', 'multiplicador': 1}, 'children': [], 'nota': 

In [51]:
def unnest_ind(df, drop='fonte'):
    """Returns an unnested dataframe with respect to the 'children' col
    for the 'indicadores' of a 'pesquisa'
    
    Parameters
    ----------
    df : DataFrame
        Orininal dataframe containing nested children data
    drop : array or string, optional
        Columns to be dropped by the end of the processing;
        By default, drops 'fonte'

    Returns
    -------
    DataFrame
        The processed DataFrame
    """
        
    df_all = df.copy()
    
    while df_all['children'].isna().sum() < len(df_all):
        for i in range(len(df_all)):
            try:
                children_df = pd.concat([pd.DataFrame(json_normalize(x)) for x in df_all.loc[i,'children']],
                                        ignore_index=True, sort=False)
                df_all = df_all.append(children_df, sort=False)
            except  Exception as e:
                pass
            finally:
                df_all.loc[i,'children'] = None
        df_all = df_all.reset_index(drop=True)
    
    df_all = df_all.drop(columns='children')
    
    try:
        df_all = df_all.drop(columns=drop)
    except:
        pass

    return df_all

In [28]:
df_ind_un = unnest_ind(df_ind.copy())

In [29]:
df_ind_un

,classe,id,indicador,nota,posicao,unidade.classe,unidade.id,unidade.multiplicador
0,T,60027,Ambiente,[],1,NaN,NaN,NaN
1,T,60046,Economia,[],2,NaN,NaN,NaN
2,T,60039,Educação,[],3,NaN,NaN,NaN
3,T,60028,Saúde,[],4,NaN,NaN,NaN
4,T,60035,Trabalho,[],5,NaN,NaN,NaN
5,I,60029,Arborização de vias públicas,"[{'periodo': '2010', 'notas': ['[domicílios urbanos em face de quadra com arborização/domicílios urbanos totais] x100']}]",1.1,N,%,1.0
6,I,60030,Esgotamento sanitário adequado,"[{'periodo': '2010', 'notas': ['[população total residente nos domicílios particulares permanentes com esgotamento sanitário do tipo rede geral e fossa séptica / População total residente nos domicílios particulares permanentes] x 100']}]",1.2,N,%,1.0
7,I,60031,Urbanização de vias públicas,"[{'periodo': '2010', 'notas': ['[domicílios urbanos em face de quadra com boca de lobo e pavimentação e meio-fio e calçada/domicílios urbanos totais] x 100']}]",1.3,N,%,1.0
8,I,60047,PIB per capita,[],2.1,$,R$,1.0
9,I,60048,Percentual das receitas oriundas de fontes externas,[],2.2,N,%,1.0


In [30]:
df_val_ind = df_val.merge(df_ind_un)

In [31]:
df_val_ind.tail()

,localidade,id,ano,valor,classe,indicador,nota,posicao,unidade.classe,unidade.id,unidade.multiplicador
86,350010,60038,2013,None,I,Salário médio mensal,[],5.3,N,salários mínimos,1.0
87,350010,60038,2014,2.2,I,Salário médio mensal,[],5.3,N,salários mínimos,1.0
88,350010,60038,2015,2.1,I,Salário médio mensal,[],5.3,N,salários mínimos,1.0
89,350010,60038,2016,None,I,Salário médio mensal,[],5.3,N,salários mínimos,1.0
90,350010,60038,2017,None,I,Salário médio mensal,[],5.3,N,salários mínimos,1.0


In [32]:
len(df_val_ind)

91

## Juntar todos os passos

Agregar todos os passos acima em uma única etapa de pré-processamento.

In [59]:
def get_unnested_ind(pes):
    """Helper function. 
    
    Returns unnested DataFrame with 'indicadores' for given 'pesquisa'
    
    Parameters
    ----------
    pes : string
        String composed of one or more 'pesquisa' identifiers, separated by |
        (pipe)

    Returns
    -------
    DataFrame
        The processed DataFrame
    """
    
    url = 'https://servicodados.ibge.gov.br/api/v1/pesquisas/{}/indicadores/0'
    df = get_df_from_endpoint(url, [pes])
    df = unnest_ind(df)
    return df

def get_expanded_res(pes,loc):
    """Helper function. 
    
    Returns expanded DataFrame with 'resultados' for given 'pesquisa' 
    and 'localidade'
    
    Parameters
    ----------
    pes : string
        String composed of one or more 'pesquisa' identifiers, separated by |
        (pipe)
    loc : string
        String composed of one or more 'localidade' identifiers, separated by |
        (pipe)

    Returns
    -------
    DataFrame
        The processed DataFrame
    """
    
    url = 'https://servicodados.ibge.gov.br/api/v1/pesquisas/{}/resultados/{}'
    df = get_df_from_endpoint(url,[pes,loc],normalize=True)
    df = expand_res(df)
    return df

def get_denorm_res_ind(pes,loc):
    """Helper function. 
    
    Returns merged DataFrame created from the expanded 'resultados' 
    (for given 'pesquisa' and 'localidade') and unnested 'indicadores'
    (for given 'pesquisa')
    
    
    Parameters
    ----------
    pes : string
        String composed of one or more 'pesquisa' identifiers, separated by |
        (pipe)
    loc : string
        String composed of one or more 'localidade' identifiers, separated by |
        (pipe)

    Returns
    -------
    DataFrame
        The processed DataFrame
    """
    
    df_ind = get_unnested_ind(pes)
    df_res = get_expanded_res(pes,loc)
    df_res = df_res.merge(df_ind, sort=False)
    return df_res

In [60]:
id_pes = 10058
id_mun = mun_to_id['Adamantina']

df_test = get_denorm_res_ind(id_pes, id_mun)

In [61]:
df_test.tail()

,localidade,id,ano,valor,classe,indicador,nota,posicao,unidade.classe,unidade.id,unidade.multiplicador
86,350010,60038,2013,None,I,Salário médio mensal,[],5.3,N,salários mínimos,1.0
87,350010,60038,2014,2.2,I,Salário médio mensal,[],5.3,N,salários mínimos,1.0
88,350010,60038,2015,2.1,I,Salário médio mensal,[],5.3,N,salários mínimos,1.0
89,350010,60038,2016,None,I,Salário médio mensal,[],5.3,N,salários mínimos,1.0
90,350010,60038,2017,None,I,Salário médio mensal,[],5.3,N,salários mínimos,1.0


In [62]:
len(df_test)

91

In [56]:
id_mun = mun_to_id['Adamantina']
error_id_arr = []
for i in pesquisas_disponiveis:
    try:
        print('Trying id_pes = {}'.format(i))
        df_test = get_denorm_res_ind(i, id_mun)
        print('OK\n')
    except Exception as e:
        print('Error processing id_pes = {}'.format(i))
        print('{}\n'.format(e))
        error_id_arr.append(i)

Trying id_pes = 1
Error processing id_pes = 1
Bad Request for url	https://servicodados.ibge.gov.br/api/v1/pesquisas/1/indicadores/0

Trying id_pes = 13
OK

Trying id_pes = 14
OK

Trying id_pes = 15
OK

Trying id_pes = 16
OK

Trying id_pes = 17
OK

Trying id_pes = 18
OK

Trying id_pes = 19
OK

Trying id_pes = 20
OK

Trying id_pes = 21
OK

Trying id_pes = 22
OK

Trying id_pes = 23
OK

Trying id_pes = 24
Error processing id_pes = 24
Bad Request for url	https://servicodados.ibge.gov.br/api/v1/pesquisas/24/indicadores/0

Trying id_pes = 29
OK

Trying id_pes = 30
OK

Trying id_pes = 31
OK

Trying id_pes = 32
OK

Trying id_pes = 33
OK

Trying id_pes = 34
Error processing id_pes = 34
'id'

Trying id_pes = 35
OK

Trying id_pes = 36
OK

Trying id_pes = 37
OK

Trying id_pes = 38
OK

Trying id_pes = 39
OK

Trying id_pes = 40
OK

Trying id_pes = 42
OK

Trying id_pes = 10058
OK

Trying id_pes = 10075
OK

Trying id_pes = 10077
OK

Trying id_pes = 10078
OK

Trying id_pes = 10079
Error processing id_pe

In [72]:
# Can't query the following 'pesquisas'
df_pes_mun[['id','nome','contexto']].where(df_pes_mun['id'].isin(error_id_arr)).dropna()

,id,nome,contexto
0,1.0,MUNIC - Perfil dos Municípios Brasileiros,10
13,24.0,Censo Agropecuário,101010
19,34.0,Pesquisa de serviços de hospedagem,101010
61,10079.0,MUNIC - Suplemento Assistência Social,10
67,10085.0,MUNIC - Suplemento Cultura,10


In [73]:
# List of available 'pesquisas'
df_pes_mun[['id','nome','contexto']].where(~df_pes_mun['id'].isin(error_id_arr)).dropna()

,id,nome,contexto
2,13.0,Censo escolar - sinopse,101010
3,14.0,Produção Agrícola - Lavoura Temporária,101010
4,15.0,Produção Agrícola - Lavoura Permanente,101010
5,16.0,Extração vegetal e Silvicultura,101010
6,17.0,Morbidade,1010
7,18.0,Pecuária,101010
8,19.0,Cadastro Central de Empresas,101010
9,20.0,Registro civil,101010
10,21.0,Finanças públicas,1010
11,22.0,Frota de veículos,101010


### Explorando 10084

Suplemento Inclusão Produtiva

In [121]:
id_mun

3500105

In [116]:
id_pes = 10084
id_mun = mun_to_id['Adamantina']

df_sip = get_denorm_res_ind(id_pes, id_mun)

In [117]:
df_sip.head()

,localidade,id,ano,valor,classe,indicador,nota,posicao
0,350010,72144,2014,Não,I,Outros,[],3.1.1.2.8
1,350010,72143,2014,Não,I,Entidades de classe patronais,[],3.1.1.2.7
2,350010,72142,2014,Não,I,Entidades de classe de trabalhadores,[],3.1.1.2.6
3,350010,72141,2014,Não,I,"Universidades, Institutos de ensino técnico e profissional",[],3.1.1.2.5
4,350010,72140,2014,Não,I,Sistema S,"[{'periodo': '2014', 'notas': ['Sistema S - nome pelo qual ficou convencionado de se chamar ao conjunto de nove instituições de interesse de categorias profissionais, estabelecidas pela Constituição Brasileira. São elas: SENAR - Serviço Nacional de Aprendizagem Rural; SENAC - Serviço Nacional de Aprendizagem Comercial; SESC - Serviço Social do Comércio; SESCOOP - Serviço Nacional de Aprendizagem do Cooperativismo; SENAI - Serviço Nacional de Aprendizagem Industrial; SESI - Serviço Social da Indústria; SEST - Serviço Social de Transporte; SENAT - Serviço Nacional de Aprendizagem do Transporte e o SEBRAE - Serviço Brasileiro de Apoio às Micro e Pequenas Empresas.']}]",3.1.1.2.4


In [118]:
for i in df_sip['indicador'].unique():
    print(i)

Outros
Entidades de classe patronais 
Entidades de classe de trabalhadores
Universidades, Institutos de ensino técnico e profissional
Sistema S
ONGs e entidades do terceiro setor
Governo Estadual
Governo federal (PRONATEC)
Parceiros responsáveis pela implementação
Especifique
Catadores de materiais recicláveis
População em situação de rua
Beneficiários do Programa Bolsa Família
Inscritos no Cadastro Único para Programas Sociais
Desempregados
Grupos para os quais as ações estão voltadas
Governo municipal desenvolve ações de qualificação profissional
Entidades de classe patronais rurais
Entidades de classe de trabalhadores rurais
Governo Federal
Médio e/ou grande produtor rural
Outros povos e comunidades tradicionais
Comunidades remanescentes de quilombos
Pescadores artesanais e aquicultores familiares
Povos indígenas
Extrativistas e silvicultores
Assentados da reforma agrária
Agricultores familiares
Governo municipal desenvolve ações de assistência técnico-gerencial a empreendimentos co

In [119]:
df_sip[df_sip['indicador']=='Assistência social'].head()

,localidade,id,ano,valor,classe,indicador,nota,posicao
197,350010,71906,2014,Sim,I,Assistência social,[],1.2.2.2


In [120]:
# TODO remove trailing white spaces
df_sip[df_sip['indicador']=='Reciclagem '].head()

,localidade,id,ano,valor,classe,indicador,nota,posicao
270,350010,72207,2014,Não,I,Reciclagem,"[{'periodo': '2014', 'notas': ['Reciclagem de resíduos sólidos e óleos/gorduras residuais.']}]",3.4.1.1.29


### Explorando pesquisa 33

Sinopse municipal

In [74]:
id_pes = 33
id_mun = mun_to_id['Adamantina']

df_sin = get_denorm_res_ind(id_pes, id_mun)

In [77]:
df_sin.dropna(how='any').head()

,localidade,id,ano,valor,classe,indicador,nota,posicao,unidade
3,350010,29171,2017,35139,I,População estimada,"[{'periodo': '2019', 'notas': ['Para ""dúvidas e contestações"" [clique aqui](https://www.ibge.gov.br/estatisticas/sociais/populacao/9103-estimativas-de-populacao.html)']}]",1,"{'id': 'pessoas', 'classe': 'N', 'multiplicador': 1}"
4,350010,29171,2018,35023,I,População estimada,"[{'periodo': '2019', 'notas': ['Para ""dúvidas e contestações"" [clique aqui](https://www.ibge.gov.br/estatisticas/sociais/populacao/9103-estimativas-de-populacao.html)']}]",1,"{'id': 'pessoas', 'classe': 'N', 'multiplicador': 1}"
5,350010,29171,2019,35068,I,População estimada,"[{'periodo': '2019', 'notas': ['Para ""dúvidas e contestações"" [clique aqui](https://www.ibge.gov.br/estatisticas/sociais/populacao/9103-estimativas-de-populacao.html)']}]",1,"{'id': 'pessoas', 'classe': 'N', 'multiplicador': 1}"
6,350010,29166,2010,33797,I,População,[],2,"{'id': 'pessoas', 'classe': 'N', 'multiplicador': 1}"
15,350010,29167,2017,411.987,I,Área da unidade territorial,"[{'periodo': '2016', 'notas': ['Disponível em: <http://www.ibge.gov.br/home/geociencias/cartografia/default_territ_area.shtm>. Acesso em: jun. 2017']}]",3,"{'id': 'km²', 'classe': 'N', 'multiplicador': 1}"


In [78]:
for i in df_sin['indicador'].unique():
    print(i)

População estimada
População
Área da unidade territorial
Densidade demográfica
Código do Município
Prefeito
Gentílico


### Explorando 43

Panorama municipal

In [101]:
id_pes = 43
id_mun = mun_to_id['Adamantina']

df_pan = get_denorm_res_ind(id_pes, id_mun)

In [102]:
df_pan.head()

,localidade,id,ano,valor,classe,indicador,nota,posicao
0,350010,30281,1970,31798,I,População residente,[],1
1,350010,30281,1980,32049,I,População residente,[],1
2,350010,30281,1991,None,I,População residente,[],1
3,350010,30281,1996,32331,I,População residente,[],1
4,350010,30281,2000,33497,I,População residente,[],1


In [103]:
for i in df_pan['indicador'].unique():
    print(i)

População residente
Nº de domicílios


### Explorando pesquisa 32

Serviços de saúde

In [79]:
id_pes = 32
id_mun = mun_to_id['Adamantina']

df_sau = get_denorm_res_ind(id_pes, id_mun)

In [82]:
df_sau.head()

,localidade,id,ano,valor,classe,indicador,nota,posicao,unidade,unidade.classe,unidade.id,unidade.multiplicador
0,350010,28163,2005,21,I,Estabelecimentos de Saúde,[],1,"{'id': 'estabelecimentos', 'classe': 'N', 'multiplicador': 1}",NaN,NaN,NaN
1,350010,28163,2009,26,I,Estabelecimentos de Saúde,[],1,"{'id': 'estabelecimentos', 'classe': 'N', 'multiplicador': 1}",NaN,NaN,NaN
2,350010,28178,2005,1,I,Com internação,[],1.2.1.1.1,NaN,N,estabelecimentos,1.0
3,350010,28178,2009,1,I,Com internação,[],1.2.1.1.1,NaN,N,estabelecimentos,1.0
4,350010,28180,2005,0,I,Público,[],1.2.1.1.1.1.1,NaN,N,estabelecimentos,1.0


In [84]:
for i in df_sau['indicador'].unique():
    print(i)

Estabelecimentos de Saúde
Com internação
Público
Privado
SUS
Sem internação
Com fins lucrativos
Sem fins lucrativos
Particular
Plano de terceiros
Plano próprio
SUS Diálise
SUS Emergência
SUS UTI/CTI
Atendimento ambulatorial
Sem atendimento médico
Com especialidades médicas básicas
Com atendimento odontológico
Serviços de apoio à diagnose e terapia - SADT
Único
Com terceirização
Eletroencefalógrafo
Equipamento de hemodiálise
Com estéreo-taxia
Até 100mA
Mais de 500mA
Ressonância magnética
Tomógrafo
Ultrassom doppler colorido
Número de leitos para internação em estabelecimentos de saúde
Federal
Estadual
Municipal
Raio X para densitometria óssea
100 a 500mA
Com comando simples
Eletrocardiógrafo
Terceirizado
Com outras especialidades médicas
SUS Internação
SUS Ambulatorial
Atendimento de emergência
